In [1]:
from hdfs import InsecureClient
from pyspark.sql import SparkSession
import json
from elasticsearch import Elasticsearch,helpers

In [2]:
#Setting up Elasticsearch Client
es = Elasticsearch(hosts="elasticsearch:9200",http_auth=("elastic","elastic@1234"))

### Listing all files stored at the /dfs-data location

In [3]:
client = InsecureClient("http://hadoop-filebeat:50070",user="nifi")
client.list('/dfs-data')

### Initializing Spark Session object

In [5]:
spark = (SparkSession
        .builder
        .appName("hdfsprocessor")
        .config("spark.jars","/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar")
        .config("spark.jars","/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-spark-20_2.11-7.11.1.jar") 
        .config("spark.es.nodes","elasticsearch")
        .config("spark.es.port","9200")
        .getOrCreate())

### Inserting documents into Elasticsearch cluster

In [1]:
# Initializing a list to insert into elasticsearch
final_json_list = []

In [11]:
for file in client.list('/dfs-data'):
    df_load = spark.read.format("csv").option("delimiter", ",").option("header", "false").load(f"hdfs://hadoop-filebeat:8020/dfs-data/{file}")
    final_df = df_load.select(df_load._c0.alias('collision_id'), df_load._c1.alias('crash_date'), df_load._c2.alias('crash_time'), df_load._c3.alias('borough'), df_load._c4.alias('zip_code'), df_load._c5.alias(
    'latitude'), df_load._c6.alias('longitude'), df_load._c7.alias('on_street_name'), df_load._c9.alias('number_of_persons_injured'), df_load._c10.alias('number_of_persons_killed'), df_load._c19.alias('vehicle_type'))
#     final_df.coalesce(1).write.format("csv").save(f"hdfs://hadoop-filebeat:8020/csv-data/{file}.csv") #
    json_ = final_df.toJSON().map(lambda j: json.loads(j)).collect()
    final_json_list.extend(json_)

In [13]:
helpers.bulk(
        es,
        final_json_list,
        index='elk_project',
        doc_type='_doc')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(15, [])